# <span style="color:red"> NUMERICAL EXERCISES 08 </span>

###### QUANTUM VARIATIONAL MC

- This JN aims to find a good approximation of the ground state $\Psi_0$ for the Hamiltonian

$$
\hat H = -\frac{\hbar^2}{2m}\Delta + V(x)
$$

   where $V(x)=x^4-\frac 5 2 x^2$ and $\hbar=m=1$.


- A good approximation is done by searching for the best trial function of the form

$$
\Psi_T^{\sigma,\mu}(x) \propto e^{-\frac{(x-\mu)^2}{2\sigma^2}}+
                               e^{-\frac{(x+\mu)^2}{2\sigma^2}}
$$

   where $\sigma,\mu$ are (positive) tunable parameters.

- Optimization in the space of these functions is achieved by exploiting the variational principle

$$
\langle {\hat H} \rangle_T = 
\frac{\int dx \Psi^*_T(x) {\hat H} \Psi_T(x)}
{\int dx |\Psi_T(x)|^2} \ge E_0 =
\frac{\langle \Psi_0| {\hat H} | \Psi_0 \rangle}
{\langle \Psi_0 | \Psi_0 \rangle}
$$

- The eigenfunctions I will find will be compared with those obtained from the diagonalization of the discretized Hamiltonian (discretization = continuous --> 1000 points in [-5.5]; laplacian --> discretized laplacian).


- Taking advantage of the parity of the potential and using fundamental quantum mechanical arguments, one could find the first excited state $\Psi_1$, which will be an odd function, using the variational principle on the space of functions (with node in the origin) of the form

$$
\Psi_T^{\sigma,\mu}(x) \propto e^{-\frac{(x-\mu)^2}{2\sigma^2}}-
                               e^{-\frac{(x+\mu)^2}{2\sigma^2}}
$$

## <span style="color:blue">Exercise 08.1 </span>

###### MC INTEGRAL

First of all I need a c++ code that involves calculating the integral $\langle {\hat H} \rangle_T$, fixed $\sigma,\mu$, using the Metropolis algorithm.

Manipulating the exepression of the integral yields the following expression:

$$
\langle {\hat H} \rangle_T = 
\frac{\int dx \Psi^*_T(x) {\hat H} \Psi_T(x)}
{\int dx |\Psi_T(x)|^2} = 
\int dx \frac{|\Psi_T(x)|^2}{\int dx |\Psi_T(x)|^2} \frac{{\hat H} \Psi_T(x)}{\Psi_T(x)}  \dot= 
{\int dx \rho(x) f(x)}
$$

The integral is written as the integral of a probability distribution $\rho$ times a function $f$, so once I can sample $\rho$, I can solve the integral by the importance sampling method.

To perform the sampling I exploit the Metropolis algorithm . Given a point $x_n$ extracted from the distribution, I can extract the next $x_{n+1}$ by proposing a point $y$ generated with uniform probability in $[x_n-walk, x_n+walk]$. $walk$ is chosen so as to satisfy the 50/50 rule. The acceptance probability of $y$ will take a simple form because we simplify both the transition probabilities (due to their symmetry) and rho normalizations (which appear in numerator and denominator). If the move is accepted, then $x_{n+1}=y$, else $x_{n+1}=x_n$. 

I don't have to wait for equilibration because I choose zero as the starting point, which is a very likely point and therefore close to equilibrium.

The graph below represent the progressive value of $\langle {\hat H} \rangle_T$ (with the a posteriori values $\mu =0.8$, $\sigma=0.6$) with error bars, as a function of the number of blocks $N$ ($M=1,000,000$, $N=100$, $L=10,000$).

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

M=1000000              # Total number of throws
N=100                 # Number of blocks
L=int(M/N)            # Number of throws in each block, please use for M a multiple of N

x = np.arange(N)      # [0,1,2,...,N-1]

sum_prog, err_prog = np.loadtxt("data/integrale.txt", usecols =(1,2), unpack = 'true')
    
x*=L # Number of throws = block * (Number of throws in each block)

plt.errorbar(x,sum_prog,yerr=err_prog)
plt.title('Integral with mu=0.8, sigma=0.6')
plt.xlabel('#throws')
plt.ylabel('value of integral')
plt.grid(True)

plt.show()

## <span style="color:blue">Exercise 08.2 </span>

###### SIMULATED ANNEALING

To perfrom the optimization of $\mu$ and $\sigma$ parameters, I have used the Simulated Annealing (SA) algorithm.
The SA algorithm is based on the analogy with the formation of crystal lattices: a fast-frozen molten metal will become an amorphous crystal (local minimization), while a slow cooling, ideally passing through infinite states of thermal equilibrium, will prevent the occurrence of local defects (global minimization).
The idea is to do an embedding of the present optimization problem in the Gibbs ensemble, so that it can be solved with the Metropolis algorithm, using the identifications: ($\mu$, $\sigma$)=state, $\langle {\hat H} \rangle_{\mu, \sigma} \dot=F(\mu,\sigma)$=energy, minimizing state=lower-energy state.

The implementation of SA is as follows:
1) set random initial values of parameters in the range of interest (e.g.$\mu = \sigma=1$);\
2) set a (high) starting temperature, say $T=10$;\
3) extract new states with the Metropolis algorithm, choosing a transition probability ($\mu_{new}$ extracted uniformly in $[\mu-0.05, \mu+0.05]$ and $\sigma_{new}=r\sigma$ with $r$ extracted uniformly in $[0.95, 1.05]$);\
4) once the Metropolis converges to the equilibrium distribution, lower the temperature and return to step 1).


The algorithm leaves at least three sensitive questions open:
- How much to decrease the temperature at each SA step?
- When is thermal equilibrium reached at a fixed temperature?
- When does the algorithm terminate?

For the first question, I chose a gemoetric trend of temperature: at each step it decreased by multiplying by $R=0.75$.\
The second question is tricky: I should check that equilibration is achieved at each temperature, for example, by looking by eye when the graph of $F(\mu,\sigma)$ stabilizes itself. This check will be done a posteriori on the final graphs. The choice is to generate 100 states for each temperature, trusting that no equilibration is needed, since:\
A-the initial state is immediately in equilibrium (it is at almost infinite temperature);\
B-each state starts near equilibrium since it has similar temperature to the previous one.\
For the third question, I choose to stop when $T<0.0001$ (about 40 steps): at this value, in fact, the probability of accepting moves becomes practically zero: frozen system).

The final values of $\sigma$ and $\mu$ will be those that minimize the mean value of the Hamiltonian. The uncertainties with which I will know these two values will not be too much less than the individual steps in which I varied them ($0.05$ for mu, $0.05 \sigma = 0.03$ for sigma).

__________________________________________________________________________________________________________________________

I made 4 graphs:

1) Progressive estimates, with error bars, of $F(\mu, \sigma)$ as a function of the steps in the SA (so one value per temperature);\
2) Path plots of the $\mu$ (x-axis) and $\sigma$ (y-axis) minimizing parameters for each step of the SA;\
(these two plots are used to summarize the SA and visualize that the choice to stop at T=0.0001 is sensible because mu and sigma do not move more)\
3) Progressive estimation with error of $\langle {\hat H} \rangle_{\mu, \sigma}$  as a function of the number of blocks, for the best $\mu$ and $\sigma$ parameters (exercise code 08.1 is used);\
4) Plot of $|\Psi(x)|^2$ for the ground state comparing: theoretical curve with $\mu$ and $\sigma$ minimizing, curve given by solution of discretized problem, curve given by the sampling.

###### BEST VALUES

$\mu = 0.809$\
$\sigma = 0.619$\
$F(\mu, \sigma) = -0.445$

###### GRAPHS

1) Progressive estimates, with error bars, of $F(\mu, \sigma)$ as a function of the steps in the SA (so one value per temperature)

We can't see the error bars because they are too small, but we see that the value stabilizes in the latter stages of the SA

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

n=41                      # Total number of SA steps (temperatures)

x = np.arange(n)      # [0,1,2,...,N-1]
y, err = np.loadtxt("data/energiaSA.txt",usecols=(0,1), unpack = 'true')
 
plt.errorbar(x,y, yerr=err)
plt.title('Stabilization of mean value of F(mu,sigma)')
plt.xlabel('SA steps')
plt.ylabel('<H>')
plt.grid(True)

plt.show()

2) Path plots of the $\mu$ (x-axis) and $\sigma$ (y-axis) minimizing parameters for each step of the SA + stabilization of $\mu$ and $\sigma$.

Similar to the previous. 

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

n=41                      # Total number of SA steps (temperatures)

a=np.arange(n)
x,y = np.loadtxt("data/parametriSA.txt",usecols=(2,3), unpack = 'true')

plt.plot(x,y)
plt.title('Path of mu and sigma in SA')
plt.xlabel('mu')
plt.ylabel('sigma')
plt.grid(True)

plt.figure()
plt.plot(a,x)
plt.title('Stabilization of mu')
plt.xlabel('SA steps')
plt.ylabel('mu')
plt.grid(True)

plt.figure()
plt.plot(a,y)
plt.title('Stabilization of sigma')
plt.xlabel('SA steps')
plt.ylabel('sigma')
plt.grid(True)

plt.show()

3) Progressive estimation with error of $\langle {\hat H} \rangle_{\mu, \sigma}$  as a function of the number of blocks, for the minimizing $\mu$ and $\sigma$ parameters.

$M=1,000,000$, $N=100$, $L=10,000$

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np

M=1000000              # Total number of throws
N=100                 # Number of blocks
L=int(M/N)            # Number of throws in each block, please use for M a multiple of N

x = np.arange(N)      # [0,1,2,...,N-1]

sum_prog, err_prog = np.loadtxt("data/integrale.txt", usecols =(1,2), unpack = 'true')
    
x*=L # Number of throws = block * (Number of throws in each block)

plt.errorbar(x,sum_prog,yerr=err_prog)
plt.title('Integral with best values of mu and sigma')
plt.xlabel('#throws')
plt.ylabel(' Integral')
plt.grid(True)

plt.show()

4) Plot of $|\Psi(x)|^2$ for the ground state comparing: theoretical curve with $\mu$ and $\sigma$ minimizing, curve given by solution of discretized problem, curve given by the sampling

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


#1) Discrtetized problem: see exercise guide

def Vpot(x):
    return (x**2 - 2.5)*x**2
    #return 0.5*x**2

hbar = 1
m = 1
a = 10
N = 1000 # number of iterations

# Step sizes
x = np.linspace(-a/2, a/2, N)
dx = x[1] - x[0] # the step size
V = Vpot(x)

# The central differences method: f" = (f_1 - 2*f_0 + f_-1)/dx^2

CDiff = np.diag(np.ones(N-1),-1)-2*np.diag(np.ones(N),0)+np.diag(np.ones(N-1),1)
# np.diag(np.array,k) construct a "diagonal" matrix using the np.array
# The default is k=0. Use k>0 for diagonals above the main diagonal, 
# and k<0 for diagonals below the main diagonal

# Hamiltonian matrix
H = (-(hbar**2)*CDiff)/(2*m*dx**2) + np.diag(V)

# Compute eigenvectors and their eigenvalues
E,psi = np.linalg.eigh(H)

# Take the transpose & normalize
psi = np.transpose(psi)
psi = psi/np.sqrt(dx)

print("Ground state energy: ", E[0])

plt.figure(figsize=(8,5))
scale = 0.3
plt.plot(x,(psi[0])**2, label='discretized problem')


#2) Sampling curve

x,y=np.loadtxt("data/histo_best.txt", usecols =(0,1), unpack = 'true')
plt.plot(x,y, label='sampling curve')

#3) Curve with best values (not normalized)

def f(x,mu,sigma):
    a1=np.exp(-(x+mu)**2/(2*sigma**2))
    a2=np.exp(-(x-mu)**2/(2*sigma**2))
    return (a1+a2)**2
mu=0.809
sigma=0.619

x = np.linspace(-a/2, a/2, N)
y = 0.5*f(x,mu,sigma)
plt.plot(x,y, label='un-normalized theoretical curve')


#PLOT
plt.title("Best wavefunction")
plt.xlabel("x")
plt.grid(True)
plt.xlim((-3,3))
plt.ylim((-0.6,0.6))
plt.legend()
plt.show()

##### CONCLUSIONS

The final values found are:
$\mu = 0.809$, 
$\sigma = 0.619$, 
$F(\mu, \sigma) = -0.445$.

1) Simulated Annealing worked well: I started from a random state and by cooling the system more and more I arrived at an equilibrium configuration that minimizes energy.

2) Actually the minimum found is slightly higher than the minimum of the exact discretized problem ($-0.460$). This may be due to a double uncertainty and a systematic error. The uncertaintes are one in determining the best values of mu and sigma, and the other in calculating the integral given these values. The systematic error lies in the fact that I arbitrarily chose the space of wave functions on which I applied the variational principle, and this space is much smaller than the natural space where to look for the ground state, i.e., $L^2(\mathbb{R})$.

3) I could therefore improve my results by working on these two uncertainties: for the former I could lower the final temperature, increase the moves for each temperature, and vary the parameters less (try to perform slower cooling); for the latter I could increase the number of blocks and their length in the calculation of the integral. Another way to improve my results is to look for a more expressive form for the wavefunction, which is closer to the true solution of the problem.